In [17]:
library(tidyverse)
# Set the seed. Don't remove this!
set.seed(4569) 
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0     ✔ yardstick    1.0.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks readr::spec()
✖ recipes::step()   masks stats::step()
• Use suppressPackageStartupMessages() to eliminate package startup messages



In [6]:
music_data <- read_csv("features_30_sec.csv")
head(music_data)

Rows: 1000 Columns: 60
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (2): filename, label
dbl (58): length, chroma_stft_mean, chroma_stft_var, rms_mean, rms_var, spec...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,⋯,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
blues.00000.wav,661794,0.3500881,0.08875657,0.13022792,0.002826696,1784.166,129774.1,2002.449,85882.76,⋯,52.42091,-1.6902146,36.52407,-0.4089792,41.59710,-2.3035226,55.06292,1.2212907,46.93604,blues
blues.00001.wav,661794,0.3409136,0.09498025,0.09594781,0.002372739,1530.177,375850.1,2039.037,213843.76,⋯,55.35640,-0.7311246,60.31453,0.2950729,48.12060,-0.2835180,51.10619,0.5312165,45.78628,blues
blues.00002.wav,661794,0.3636372,0.08527520,0.17557041,0.002745916,1552.812,156467.6,1747.702,76254.19,⋯,40.59877,-7.7290926,47.63943,-1.8164070,52.38214,-3.4397204,46.63966,-2.2312584,30.57302,blues
blues.00003.wav,661794,0.4047847,0.09399904,0.14109300,0.006346346,1070.107,184355.9,1596.413,166441.49,⋯,44.42775,-3.3195965,50.20667,0.6369650,37.31913,-0.6191212,37.25974,-3.4074483,31.94934,blues
blues.00004.wav,661794,0.3085260,0.08784098,0.09152871,0.002303398,1835.004,343399.9,1748.172,88445.21,⋯,86.09924,-5.4540339,75.26971,-0.9168741,53.61392,-4.4048271,62.91081,-11.7032337,55.19516,blues
blues.00005.wav,661794,0.3024563,0.08753238,0.10349364,0.003980508,1831.994,1030482.4,1729.653,201910.51,⋯,72.54922,-1.8382628,68.70203,-2.7837999,42.44745,-3.0479090,39.80878,-8.1099911,46.31100,blues


In [21]:
music_data <- music_data |> filter(label %in% c("jazz", "rock", "pop", "classical"))

split_data <- initial_split(music_data, props = 0.75, strata = label)
music_train <- training(split_data)
music_test <- testing(split_data)

summary <- music_train |> group_by(label) |> summarize(count = n(), across(chroma_stft_mean:mfcc20_var, mean))

summary

sum
<dbl>
-0.0926986


label,count,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,⋯,mfcc16_mean,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var
<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
classical,75,0.2651851,0.08506837,0.04785598,0.0007934635,1392.688,133502.3,1530.005,79271.49,⋯,1.2606485,77.24471,0.05952114,77.25029,0.55616208,81.91501,-1.5324562,95.93516,-1.3574386,114.07549
jazz,75,0.2887632,0.09022069,0.08752246,0.0015375521,1711.513,260998.9,1947.915,106158.15,⋯,0.6630296,85.97275,-1.41176639,85.35002,-0.71787167,88.31595,-1.0537729,95.84098,-1.9173392,108.05283
pop,75,0.4000467,0.08893888,0.19713133,0.0074959087,3095.316,935355.0,3008.801,175839.16,⋯,-1.0458814,74.63859,-0.47814423,77.89647,0.22058179,81.32177,0.7457283,83.19417,0.2046489,84.65399
rock,75,0.3876597,0.08439164,0.12658217,0.0017573213,2194.351,425930.4,2236.538,132942.90,⋯,0.9527229,43.55403,-6.70909722,47.11607,0.03696864,48.55023,-4.0445457,47.93153,-1.7149802,48.41267
